In [1]:
import pandas as pd
import requests
import numpy as np
from pandas import json_normalize
from pathlib import Path
from api_key import api_key

In [2]:
# Station codes as strings
station_code_list = ["BG001"]

In [3]:
# Headers with API key
headers = {
    "api_key": api_key
}

In [5]:
# Initialize an empty list to store the data for all stations
all_stations_data = []

# Number of iterations
num_iterations = 600

# Iterate through each station
for station in station_code_list:
    # Iterate over the specified number of times
    for i in range(num_iterations):
        # Parameters for the specific station
        url = f"https://api.dpird.wa.gov.au/v2/weather/stations/{station}/data?"
        params = {
            "startDateTime": "2023-07-01T00:00:00Z",
            "endDateTime": "2024-01-01T00:00:00Z",
            "offset": i * 1000,  # Use the current iteration index as the offset
            "limit": 1000,
        }

        # Make the request for each station
        response = requests.get(url, params=params, headers=headers)
        data = response.json()

        # Check if the 'collection' list is non-empty
        if "collection" in data:
            # Extract relevant information for each summary
            for summary in data["collection"]:
                try:
                    station_info = {
                        "dateTime": summary["dateTime"],
                        "airTemperature": summary["airTemperature"],
                        "relativeHumidity": summary["relativeHumidity"],
                        "soilTemperature": summary["soilTemperature"],
                        "solarIrradiance": summary["solarIrradiance"],
                        "rainfall": summary["rainfall"],
                        "dewPoint": summary["dewPoint"],
                        "deltaT": summary["deltaT"],
                        "wetBulb": summary["wetBulb"],
                        "wind_avg_speed": summary["wind"][0]["avg"]["speed"],
                        "wind_direction": summary["wind"][0]["avg"]["direction"]["compassPoint"],
                        "barometricPressure": summary["barometricPressure"],
                        "batteryVoltage": summary["batteryVoltage"],
                    }
                except KeyError as e:
                    # Handle specific exception (e.g., key not found in the response)
                    # Use a default station_info with all values as np.nan
                    station_info = {
                        "dateTime": np.nan,
                        "airTemperature": np.nan,
                        "relativeHumidity": np.nan,
                        "soilTemperature": np.nan,
                        "solarIrradiance": np.nan,
                        "rainfall": np.nan,
                        "dewPoint": np.nan,
                        "deltaT": np.nan,
                        "wetBulb": np.nan,
                        "wind_avg_speed": np.nan,
                        "wind_direction": np.nan,
                        "barometricPressure": np.nan,
                        "batteryVoltage": np.nan,
                    }

                # Append the information for the current station and summary to the list
                all_stations_data.append(station_info)

In [6]:
# Assuming all_stations_data is a list of dictionaries
all_stations = pd.DataFrame(all_stations_data)

# Display dataframe
all_stations

,dateTime,airTemperature,relativeHumidity,soilTemperature,solarIrradiance,rainfall,dewPoint,deltaT,wetBulb,wind_avg_speed,wind_direction,barometricPressure,batteryVoltage
0,2023-01-01T00:00:00Z,19.1,54.8,24.1,573,0.0,9.8,4.8,14.3,7.70,SE,None,14.36
1,2023-01-01T00:01:00Z,19.2,55.2,24.1,577,0.0,10.0,4.7,14.5,6.19,SE,None,14.35
2,2023-01-01T00:02:00Z,19.2,54.4,24.1,583,0.0,9.8,4.8,14.4,7.20,SE,None,14.36
3,2023-01-01T00:03:00Z,19.2,54.3,24.2,587,0.0,9.8,4.9,14.3,5.18,ESE,None,14.37
4,2023-01-01T00:04:00Z,19.2,53.8,24.2,592,0.0,9.6,4.9,14.3,8.21,SSE,None,13.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
260630,2023-06-30T23:56:00Z,6.8,72.7,7.3,22,0.0,2.3,1.8,5.0,2.16,ENE,None,12.50
260631,2023-06-30T23:57:00Z,6.9,72.7,7.3,23,0.0,2.3,1.8,5.1,4.68,NE,None,12.57
260632,2023-06-30T23:58:00Z,6.8,72.6,7.3,24,0.0,2.2,1.8,5.0,0.65,NNE,None,12.58
260633,2023-06-30T23:59:00Z,6.8,72.7,7.2,23,0.0,2.3,1.8,5.0,1.15,NE,None,12.59


In [7]:
all_stations.to_csv("../Resources/raindata.csv", index=False)